In [1]:
import numpy as np
import random as rnd
import copy

In [2]:
# define the environment and agent paramters
num_of_col = 5
num_of_row = 4
num_of_element = num_of_col * num_of_row
regular_reward_value = 0
goal_reward_value = 100
goal_col_index = 5
goal_row_index = 4
start_col_index = 1
start_row_index = 1

gamma = 0.9 # discount rate

In [3]:
# Generate the reward matrix in the grid
# Grid position (i, j) is (col, row), 1-based index, start from (1, 1)
# The state index is computed by the formula: 
# index = (j - 1) * num_of_columns + (i - 1) with (i, j) represent ith column and jth row in the grid
# Example:
# Grid environment with four rows and five column
# Goal state is at (5, 4) whose state index is (4-1)*5 + (5 - 1) = 19
# Start state is at (1,1) whose state index is (1-1)*5 + (1 - 1) = 0

class GridStateMatrix:
    def __init__(self, num_of_col, num_of_row, default_value=0):
        self.num_of_col = num_of_col # number of columns of the grid environment (position)
        self.num_of_row = num_of_row # number of rows of the grid environment (position)
        self.num_of_state = self.num_of_col * self.num_of_row # number of states in the state-action matrix
        self.default_value = default_value # default value for initial matrix
        matrixValues = [self.default_value for _ in range(self.num_of_state * self.num_of_state)]
        self.gridStateMatrix = np.matrix(matrixValues).reshape((self.num_of_state, self.num_of_state))
        
    def get_state_index(self, col_index, row_index):
        return (row_index - 1) * num_of_col + (col_index - 1)
    
    def get_grid_position(self, state_index):
        if (state_index >= self.num_of_state):
            return -1, -1
        row_index = state_index // self.num_of_col + 1
        col_index = state_index % self.num_of_col + 1
        return col_index, row_index
    
    def get_neighbors(self, col_index, row_index):
        neighbors = list()
        if (row_index < self.num_of_row): # up
            neighbors.append((col_index, row_index+1))
        if (row_index > 1): # down
            neighbors.append((col_index, row_index-1))
        if (col_index > 1): # left
            neighbors.append((col_index-1, row_index))
        if (col_index < self.num_of_col): # right
            neighbors.append((col_index+1, row_index))
        return neighbors
    
    def set_element_value(self, value, state_index, action_index):
        if (state_index >= self.num_of_state):
            return False
        if (action_index >= self.num_of_state):
            return False
        self.gridStateMatrix[state_index, action_index] = value
        return True

    def get_state_matrix(self):
        return self.gridStateMatrix

In [4]:
Reward = GridStateMatrix(num_of_col,num_of_row, np.inf)

# set up state-action relation between neighbor grid cell
for col in range(num_of_col):
    for row in range(num_of_row):
        neighbors = Reward.get_neighbors(col+1, row+1)
        s_index = Reward.get_state_index(col+1, row+1)
        for i, j in neighbors:
            #print col+1, row+1, '->', i, j
            a_index = Reward.get_state_index(i, j)
            Reward.set_element_value(regular_reward_value, s_index, a_index)

relations = Reward.get_neighbors(goal_col_index, goal_row_index)
a_index = Reward.get_state_index(goal_col_index, goal_row_index)
for i, j in relations:
    #print i, j
    s_index = Reward.get_state_index(i,j)
    Reward.set_element_value(goal_reward_value, s_index, a_index)
if not Reward.set_element_value(goal_reward_value, a_index, a_index):
    print 'Goal reward value setting failed!'
        
#print Reward.get_state_matrix()

In [5]:
# find the max Q value over all possible actions at the given state
def get_max_Q_action(Q, s_index, epsilon=0.0):
    s_col, s_row = Q.get_grid_position(s_index)
    neighbors = Q.get_neighbors(s_col, s_row)
    rnd.shuffle(neighbors)
    num_of_neighbors = len(neighbors)
    a_max_index = -1
    Q_max = -np.inf
    for n_col, n_row in neighbors:
        n_index = Q.get_state_index(n_col, n_row)
        #print s_col, s_row, '->', n_col, n_row
        #print s_index, '->', n_index
        Q_n = Q.get_state_matrix()[s_index, n_index]
        if (Q_n > Q_max):
            Q_max = Q_n
            a_max_index = n_index
    if (rnd.random() < epsilon and num_of_neighbors!=0):
        choice = rnd.randint(0, num_of_neighbors-1)
        a_index = Q.get_state_index(neighbors[choice][0], neighbors[choice][1])
    else:
        a_index = a_max_index
    return Q_max, a_max_index, a_index


def q_learning_process(Reward, start_state_index, goal_state_index, goal_reward_value, alpha, epsilon=0.0, total_episode=100):
    num_of_state = Reward.get_state_matrix().shape[0]
    num_of_col, num_of_row = Reward.get_grid_position(num_of_state-1)
    # initialize Q matrix and V matrix
    Q_current = GridStateMatrix(num_of_col, num_of_row)
    V_values = [0 for _ in range(num_of_col*num_of_row)]
    V_matrix = np.matrix(V_values).reshape((num_of_row, num_of_col))
    goal_col, goal_row = Q_current.get_grid_position(goal_state_index)
    V_matrix[goal_row-1, goal_col-1] = goal_reward_value
    reward_values = [0 for _ in range(num_of_col*num_of_row)]
    reward_matrix = np.matrix(reward_values).reshape((num_of_row, num_of_col))
    reward_matrix[goal_row-1, goal_col-1] = goal_reward_value
    # start learning over episodes
    episode = 0
    while (episode < total_episode):
        # Initialize the state at start state
        #print episode
        episode += 1
        s_index = start_state_index
        _, _, a_index = get_max_Q_action(Q_current, s_index, epsilon)
        # Check if current state is goal state
        while (s_index != goal_state_index):
            # Choose action from s using policy derived from Q (epsilon-greedy)
            _, _, a_index = get_max_Q_action(Q_current, s_index, epsilon)
            # Take action a, observe r and s'
            next_reward = Reward.get_state_matrix()[s_index, a_index]
            # update Q(s,a) <- Q(s,a) + alpha * [r + gamma*max_a' Q(s', a') - Q(s,a)]
            next_Q, _, _ = get_max_Q_action(Q_current, a_index)
            cur_Q = Q_current.get_state_matrix()[s_index, a_index] 
            new_Q = cur_Q + alpha*(next_reward + gamma*next_Q - cur_Q)
            Q_current.set_element_value(new_Q, s_index, a_index)
            # update V(s) <- V(s) + alpha * [r + gamma*V(s') - V(s)]
            next_col, next_row = Q_current.get_grid_position(a_index)
            cur_col, cur_row = Q_current.get_grid_position(s_index)
            next_V = V_matrix[next_row-1, next_col-1]
            cur_V = V_matrix[cur_row-1, cur_col-1]
            cur_reward = reward_matrix[cur_row-1, cur_col-1]
            V_matrix[cur_row-1, cur_col-1] = cur_V + alpha*(cur_reward + gamma*next_V - cur_V)
            # move to next state s <- s' 
            s_index = a_index
    return Q_current, V_matrix

In [6]:
# initialize learning parameters
alpha = 1.0 # learning rate
epsilon = 0.0 # explore probability threshold
total_episode = 10 # total number of training episodes
start_state_index = Reward.get_state_index(start_col_index, start_row_index)
goal_state_index = Reward.get_state_index(goal_col_index, goal_row_index)
Q_current, V_matrix = q_learning_process(Reward, start_state_index, goal_state_index, goal_reward_value, alpha, epsilon, total_episode)

In [7]:
Q_1_1,_,_ = get_max_Q_action(Q_current, Q_current.get_state_index(1,1), 0.0)
print Q_1_1

51


In [8]:
Q_3_2,_,_ = get_max_Q_action(Q_current, Q_current.get_state_index(3,2), 0.0)
print Q_3_2

72


In [9]:
print V_matrix

[[ 45   0   0  64   0]
 [ 51  57  64  72   0]
 [  0   0   0  81   0]
 [  0  72  81  90 100]]


In [10]:
def retrieve_optimal_policy(Q, start_col, start_row, goal_col_index, goal_row_index):
    action_trace = list()
    s_index = Q.get_state_index(start_col, start_row)
    g_index = Q.get_state_index(goal_col_index, goal_row_index)
    action_trace.append((start_col, start_row))
    while (s_index != g_index):
        _,_,a_index = get_max_Q_action(Q, s_index)
        action_trace.append(Q.get_grid_position(a_index))
        s_index = a_index
    return action_trace

print retrieve_optimal_policy(Q_current, start_col_index, start_row_index, goal_col_index, goal_row_index)

[(1, 1), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (5, 4)]


In [11]:
Reward2 = copy.deepcopy(Reward)
# insert another goal reward at the lower right corner (5,1)
goal2_col_index = 5
goal2_row_index = 1
goal2_reward_value = 100
relations = Reward2.get_neighbors(goal2_col_index, goal2_row_index)
a_index = Reward2.get_state_index(goal2_col_index, goal2_row_index)
for i, j in relations:
    #print i, j
    s_index = Reward2.get_state_index(i,j)
    Reward2.set_element_value(goal2_reward_value, s_index, a_index)
if not Reward2.set_element_value(goal2_reward_value, a_index, a_index):
    print 'Goal reward value setting failed!'
#print Reward2.get_state_matrix()

In [12]:
def q_learning_multigoals(Reward, start_state_index, goal_state_index_list, goal_reward_list, alpha, epsilon=0.0, total_episode=100):
    num_of_state = Reward.get_state_matrix().shape[0]
    num_of_col, num_of_row = Reward.get_grid_position(num_of_state-1)
    # initialize Q matrix
    Q_current = GridStateMatrix(num_of_col, num_of_row)
    V_values = [0 for _ in range(num_of_col*num_of_row)]
    V_matrix = np.matrix(V_values).reshape((num_of_row, num_of_col))
    reward_values = [0 for _ in range(num_of_col*num_of_row)]
    reward_matrix = np.matrix(reward_values).reshape((num_of_row, num_of_col))
    for g_index, g_reward in zip(goal_state_index_list, goal_reward_list):
        g_col, g_row = Q_current.get_grid_position(g_index)
        V_matrix[g_row-1, g_col-1] = g_reward
        reward_matrix[g_row-1, g_col-1] = g_reward
    episode = 0
    while (episode < total_episode):
        # Initialize the state at start state
        #print episode
        episode += 1
        s_index = start_state_index
        _, _, a_index = get_max_Q_action(Q_current, s_index, epsilon)
        # Check if current state is goal state
        while (s_index not in goal_state_index_list):
            # Choose action from s using policy derived from Q (epsilon-greedy)
            _, _, a_index = get_max_Q_action(Q_current, s_index, epsilon)
            # Take action a, observe r and s'
            next_reward = Reward.get_state_matrix()[s_index, a_index]
            # update Q(s,a) <- Q(s,a) + alpha * [r + gamma*max_a' Q(s', a') - Q(s,a)]
            next_Q, _, _ = get_max_Q_action(Q_current, a_index)
            cur_Q = Q_current.get_state_matrix()[s_index, a_index] 
            new_Q = cur_Q + alpha*(next_reward + gamma*next_Q - cur_Q)
            Q_current.set_element_value(new_Q, s_index, a_index)
            # update V(s) <- V(s) + alpha * [r + gamma*V(s') - V(s)]
            next_col, next_row = Q_current.get_grid_position(a_index)
            cur_col, cur_row = Q_current.get_grid_position(s_index)
            next_V = V_matrix[next_row-1, next_col-1]
            cur_V = V_matrix[cur_row-1, cur_col-1]
            cur_reward = reward_matrix[cur_row-1, cur_col-1]
            V_matrix[cur_row-1, cur_col-1] = cur_V + alpha*(cur_reward + gamma*next_V - cur_V)
            # move to next state s <- s' 
            s_index = a_index
    return Q_current, V_matrix

In [13]:
# initialize learning parameters
alpha = 1.0 # learning rate
epsilon = 0.0 # explore probability threshold
total_episode = 10 # total number of training episodes
start_state_index = Reward2.get_state_index(start_col_index, start_row_index)
goal_state_index = Reward2.get_state_index(goal_col_index, goal_row_index)
goal2_state_index = Reward2.get_state_index(goal2_col_index, goal2_row_index)
goal_state_index_list = [goal_state_index, goal2_state_index]
goal_reward_list = [goal_reward_value, goal2_reward_value]
Q_current2, V_matrix2 = q_learning_multigoals(Reward2, start_state_index, goal_state_index_list, goal_reward_list, alpha, epsilon, total_episode)
print V_matrix2

[[ 64  72  81  90 100]
 [  0   0  72   0   0]
 [  0   0   0  81  90]
 [  0   0  81  90 100]]


In [14]:
print retrieve_optimal_policy(Q_current2, start_col_index, start_row_index, goal_col_index, goal_row_index)

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (5, 2), (4, 2), (3, 2), (3, 1), (4, 1), (5, 1), (5, 2), (5, 1), (4, 1), (5, 1), (4, 1), (5, 1), (4, 1), (5, 1), (5, 2), (5, 1), (4, 1), (5, 1), (4, 1), (5, 1), (4, 1), (5, 1), (4, 1), (5, 1), (5, 2), (4, 2), (4, 1), (5, 1), (5, 2), (5, 1), (5, 2), (5, 3), (5, 4)]


In [15]:
print retrieve_optimal_policy(Q_current2, start_col_index, start_row_index, goal2_col_index, goal2_row_index)

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [16]:
Reward3 = copy.deepcopy(Reward)
# replace second goal reward by negative value at the lower right corner (5,1)
goal3_col_index = 5
goal3_row_index = 1
goal3_reward_value = -100 # replaced value
relations = Reward3.get_neighbors(goal3_col_index, goal3_row_index)
a_index = Reward3.get_state_index(goal3_col_index, goal3_row_index)
for i, j in relations:
    #print i, j
    s_index = Reward3.get_state_index(i,j)
    Reward3.set_element_value(goal3_reward_value, s_index, a_index)
if not Reward2.set_element_value(goal3_reward_value, a_index, a_index):
    print 'Goal reward value setting failed!'

In [17]:
# initialize learning parameters
alpha = 1.0 # learning rate
epsilon = 0.0 # explore probability threshold
total_episode = 10 # total number of training episodes
start_state_index = Reward3.get_state_index(start_col_index, start_row_index)
goal_state_index = Reward3.get_state_index(goal_col_index, goal_row_index)
goal3_state_index = Reward3.get_state_index(goal3_col_index, goal3_row_index)
goal_state_index_list = [goal_state_index, goal3_state_index]
goal_reward_list = [goal_reward_value, goal3_reward_value]
Q_current3, V_matrix3 = q_learning_multigoals(Reward3, start_state_index, goal_state_index_list, goal_reward_list,alpha, epsilon, total_episode)
print V_matrix3

[[  45    0    0    0 -100]
 [  51    0    0    0    0]
 [  57   64   72    0    0]
 [   0    0   81   90  100]]


In [18]:
print retrieve_optimal_policy(Q_current3, start_col_index, start_row_index, goal_col_index, goal_row_index)

[(1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (5, 4)]
